In [38]:
import numpy as np
import pandas as pd
import scipy as sp

In [39]:
# df = pd.read_excel(r"C:\Users\adevr\MITx_6.419x\vol_model_zeynep\vol_data_close.xlsx")
df = pd.read_excel(r"C:\Users\Atakan\atakan_python\MITx_6.419x\vol_model_zeynep\vol_data_close.xlsx")
df = df.iloc[::-1]
df = df.set_index("date")

df["log_return"] = np.log(df["close"] / df["close"].shift(1))
df["log_return"] = df["log_return"].fillna(0)
df["log_return_sq"] = df["log_return"]**2
# df = df.dropna()

df

,close,var,log_return,log_return_sq
date,,,,
2023-07-21,26.93720,0.000021,0.000000,0.000000e+00
2023-07-24,26.95300,NaN,0.000586,3.438386e-07
2023-07-25,26.92810,NaN,-0.000924,8.542517e-07
2023-07-26,26.94475,NaN,0.000618,3.820750e-07
2023-07-27,26.94540,NaN,0.000024,5.819262e-10
...,...,...,...,...
2024-07-16,33.08615,NaN,0.001287,1.656020e-06
2024-07-17,33.09380,NaN,0.000231,5.344782e-08
2024-07-18,33.06975,NaN,-0.000727,5.285092e-07


In [40]:
df["var"].shape[0]

251

Variance

$\sigma^2_k = \sigma^2_{k-1}

In [41]:
def log_likelihood(x, var, mu=0):  # vectorized
    return -0.5 * np.log(2*np.pi*var) - 0.5 * (x - mu)**2 / var

def negative_log_likelihood(x, var, mu=0):
    return -(-0.5 * np.log(2*np.pi*var) - 0.5 * (x - mu)**2 / var)

def sum_negative_log_likelihood(x_arr, var_arr, mu=0):
    return -(-0.5 * np.log(2*np.pi*x_arr) - 0.5 * (x - mu)**2 / var_arr).sum()

type(log_likelihood)

function

In [42]:
alpha = 0

for i in range(1, df["var"].shape[0]):
    df["var"].iloc[i] = alpha * df["log_return_sq"].iloc[i-1] + (1 - alpha) * df["var"].iloc[i-1]

df["log_likelihood"] = -negative_log_likelihood(df["log_return"], df["var"])

print(df)

               close       var  log_return  log_return_sq  log_likelihood
date                                                                     
2023-07-21  26.93720  0.000021    0.000000   0.000000e+00        4.454886
2023-07-24  26.95300  0.000021    0.000586   3.438386e-07        4.446889
2023-07-25  26.92810  0.000021   -0.000924   8.542517e-07        4.435016
2023-07-26  26.94475  0.000021    0.000618   3.820750e-07        4.445999
2023-07-27  26.94540  0.000021    0.000024   5.819262e-10        4.454873
...              ...       ...         ...            ...             ...
2024-07-16  33.08615  0.000021    0.001287   1.656020e-06        4.416367
2024-07-17  33.09380  0.000021    0.000231   5.344782e-08        4.453643
2024-07-18  33.06975  0.000021   -0.000727   5.285092e-07        4.442593
2024-07-19  33.05915  0.000021   -0.000321   1.027754e-07        4.452496
2024-07-22  32.93135  0.000021   -0.003873   1.500237e-05        4.105927

[251 rows x 5 columns]


In [43]:
-0.5 * np.log(2*np.pi*0.000021) -0.5 * 0

4.466555526915753

In [44]:
def calc_gradient(x_arr, var_arr, mu=0):
    dl_dsigma2 = -0.5 * 1/var_arr + 0.5 * (x_arr - mu)**2 / var_arr
    dsigma2_dalpha = (x_arr.shift(1))**2 - var_arr.shift(1)
    return (dl_dsigma2 * dsigma2_dalpha).mean()

In [45]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt

# Load and prepare data
df = pd.read_excel(r"C:\Users\Atakan\atakan_python\MITx_6.419x\vol_model_zeynep\vol_data_close.xlsx")
df = df.iloc[::-1]
df = df.set_index("date")
df["log_return"] = np.log(df["close"] / df["close"].shift(1))
df["log_return"] = df["log_return"].fillna(0)
df["log_return_sq"] = df["log_return"]**2

# Define the negative log-likelihood function
def negative_log_likelihood(x, var, mu=0):
    return -(-0.5 * np.log(2 * np.pi * var) - 0.5 * (x - mu)**2 / var)

# Define the objective function to minimize
def sum_negative_log_likelihood(alpha, x_arr, initial_var):
    var_arr = np.zeros_like(x_arr)
    var_arr[0] = initial_var
    for i in range(1, len(x_arr)):
        var_arr[i] = alpha * x_arr[i-1]**2 + (1 - alpha) * var_arr[i-1]
    
    nll_sum = np.sum(negative_log_likelihood(x_arr, var_arr))
    return nll_sum

df

,close,var,log_return,log_return_sq
date,,,,
2023-07-21,26.93720,0.000021,0.000000,0.000000e+00
2023-07-24,26.95300,NaN,0.000586,3.438386e-07
2023-07-25,26.92810,NaN,-0.000924,8.542517e-07
2023-07-26,26.94475,NaN,0.000618,3.820750e-07
2023-07-27,26.94540,NaN,0.000024,5.819262e-10
...,...,...,...,...
2024-07-16,33.08615,NaN,0.001287,1.656020e-06
2024-07-17,33.09380,NaN,0.000231,5.344782e-08
2024-07-18,33.06975,NaN,-0.000727,5.285092e-07


In [46]:
alpha = 0
x_arr = df["log_return"]
initial_var = 0.000021

sum_negative_log_likelihood(alpha, x_arr, initial_var)

# # Minimize the objective function with respect to alpha
# result = minimize(sum_negative_log_likelihood, x0=0.2, args=(df["log_return"].values, initial_var), bounds=[(0, 1)])
# optimal_alpha = result.x[0]
# optimal_alpha

alpha_arr = linspace

C:\Users\Atakan\AppData\Local\Temp\ipykernel_21216\2266272948.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  var_arr[i] = alpha * x_arr[i-1]**2 + (1 - alpha) * var_arr[i-1]


-989.5402390389909


**Problem:**

$\alpha^* = argmin_{\alpha} \quad L(\alpha; x_1, \cdots, x_n) $

____________

Log Likelihood:

$l_k(\sigma_k^2 ; x_1, \cdots, x_k) = -0.5 \times (ln(2 \pi) + ln(\sigma_k^2) + \frac{x_k^2}{\sigma_k^2})$

Sum of Log Likelihoods:

$ L(\alpha; x_1, \cdots, x_n) = l_1(\sigma_1^2 ; x_k) + \cdots + l_k(\sigma_k^2 ; x_k) + \cdots + l_n(\sigma_n^2 ; x_k) =  \sum_{k=1}^n l_k(\sigma_k^2 ; x_k) $

Variance recursive formula:

$\sigma_k^2 = \alpha x_{k-1}^2 + (1-\alpha) \sigma_{k-1}^2 $

________________

Derivative of $l_k(\sigma_k^2 ; x_k)$ w.r.t. $\alpha$ for any $\sigma_k^2$ s.t. $1 < k < n :$

$\Large\frac{\partial l_k}{\partial \alpha} = \frac{\partial l_k}{\partial \sigma_k^2} \times \frac{\sigma_k^2}{\partial \sigma_{k-1}^2} \times \frac{\sigma_{k-1}^2}{\partial \sigma_{k-2}^2} \times \cdots \times \frac{\sigma_{1}^2}{\partial \alpha}$

Derivative of $L(\alpha; n)$ w.r.t. $alpha$:

$\Large\frac{\partial L}{\partial \alpha} = \frac{\partial l_1}{\partial \alpha} + \frac{\partial l_2}{\partial \alpha} + \cdots + \frac{\partial l_n}{\partial \alpha} $

__________________

Gradient descent update for $\alpha$:

$\alpha \leftarrow \alpha - \eta \Large\frac{\partial L}{\partial \alpha}$